1\. Write a function that converts numbers among the bin, dec, and hex representations (bin<->dec<->hex)

In [1]:
def convert(num, to_base=10):
    """Converts a number to the specified base (2, 10, 16)."""
    num = str(num)
    if num[:2] == '0b':
        # if bin convert to int
        x = int(num, 2)
    elif num[:2] == '0x':
        # if hex convert to int
        x = int(num, 16)
    else:
        x = int(num)
        
    # now x is the int representation of num
    # regardless of the input base
    
    if to_base == 2:
        return bin(x)
    elif to_base == 16:
        return hex(x)
    elif to_base == 10:
        return x
    else:
        print(f'Conversion to base {to_base} not supported')

# test dec -> bin and bin -> dec conversion
a = 10
a_bin = convert(a, 2)
print(f'{a} --> {a_bin}')
print(f'{a_bin} --> {convert(a_bin)}\n')

# test dec -> hex and hex -> dec conversion
b = 36
b_hex = convert(b, 16)
print(f'{b} --> {b_hex}')
print(f'{b_hex} --> {convert(b_hex)}\n')

# test hex -> bin and bin -> hex conversion
b_bin = convert(b_hex, 2)
print(f'{b_hex} --> {b_bin}')
print(f'{b_bin} --> {convert(b_bin, 16)}')

10 --> 0b1010
0b1010 --> 10

36 --> 0x24
0x24 --> 36

0x24 --> 0b100100
0b100100 --> 0x24


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent) and check its validity.

In [2]:
def decode_float32_str(num, bias=127):
    """Decode the floating point number num, stored as a bitstring, according to the float32 standard."""
    # firstly 'extract' the componenets (sign, exp, mantissa)
    sign = num[0]
    exp  = num[1:9]
    mant = num[9:]
    # compute the sign
    sign = -1 if sign == '1' else 1
    # compute the exponent
    exp  = sum([2**(7-i) for i, b in enumerate(exp) if b == '1'])
    # compute the fractional part
    frac = sum([2**(-i-1) for i, b in enumerate(mant) if b == '1'])
    return sign * (1+frac) * 2**(exp-bias) # use the definition

# alternative way
def decode_float32(num, bias=127):
    """Decode the floating point number num, stored as an int32, according to the float32 standard."""
    # firstly 'extract' the componenets (sign, exp, mantissa)
    sign = (num & 0x80000000) >> 31
    exp  = (num & 0x7F800000) >> 23
    mant = num & 0x7FFFFF
    # compute the fractional part
    frac = sum([((mant>>23-i)&0x1)*(2**-i)  for i in range(1, 23)])    
    return ((-1) ** sign) * (1+frac) * 2**(exp-bias) # use the definition

a = '00000011111000000000000000000000' # should return +1.316554 x 10^-36 (taken from the lecture)
print(decode_float32_str(a))
print(decode_float32(int(a, 2)))
print()

b = '11000000101100000000000000000000' # should return -5.5 (taken from the lecture)
print(decode_float32_str(b))
print(decode_float32(int(b, 2)))

1.316553672920962e-36
1.316553672920962e-36

-5.5
-5.5


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tip**: define two variables initialized to 1 and halve/double them for a sufficient amount of times to exceed the under/over-flow limits  

In [3]:
# find the overflow limit (whitin a factor of two)
num = 1.0     # int doesn't overflow
count_mul = 0 # count the successful doubling of num
while num != num * 2: # equivalent to num != float('inf')
    num *= 2
    count_mul += 1
print(f'{count_mul} doubling to reach overflow -> {float(2**(count_mul-1))} overflow limit (within a factor of 2)')

# find the underflow limit (whitin a factor of two)
num = 1.0
count_div = 0 # count the successful division of num
while num != num / 2: # equivalent to num != 0
    num /= 2
    count_div += 1
print(f'{count_div} division to reach underflow -> {float(2**-(count_div-1))} underflow limit (within a factor of 2)')

1024 doubling to reach overflow -> 8.98846567431158e+307 overflow limit (within a factor of 2)
1075 division to reach underflow -> 5e-324 underflow limit (within a factor of 2)


4\. Write a program to determine the machine precision

**Tip**: define a new variable by adding an increasingly smaller value (in the same way as the previous problem) and check when the addition starts to have no effect on the number

In [4]:
precision = 1.0
while 1 + precision != 1:
    # keep dividing the precision until it does not affect the sum
    precision /= 2

print(1 + precision == 1)
precision *= 2
print(1 + precision == 1)
print(f'The precision found is {precision}')

True
False
The precision found is 2.220446049250313e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying the numerator and the denominator by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what has been previously obtained, and why?

(c) write a function that computes the roots of a quadratic equation accurately in all cases

In [5]:
import math

# unstable if b^2 >> 4ac
def solve_second_degree_eq(a, b, c):
    d = b**2 - 4*a*c
    x1 = .5 * (-b + math.sqrt(d)) / a # sqrt(d) is close to b (if b^2 >> 4ac) -> cancellation
    x2 = .5 * (-b - math.sqrt(d)) / a # no problems
    return x1, x2

# unstable if b^2 >> 4ac
def solve_second_degree_eq2(a, b, c):
    d = b**2 - 4*a*c
    x1 = 2*c / (-b - math.sqrt(d)) # no problems
    x2 = 2*c / (-b + math.sqrt(d)) # sqrt(d) is close to b (if b^2 >> 4ac) -> cancellation
    return x1, x2

# should be more stable 
# avoids the cancellation that may happen in the other 2 functions by combining
def solve_second_degree_eq_stable(a, b, c):
    d = b**2 - 4*a*c
    x1 = 2*c / (-b - math.sqrt(d))
    x2 = .5 * (-b - math.sqrt(d)) / a
    return x1, x2

a = 1e-3
b = 1e3
c = 1e-3
print(f'equation: {a}x^2 + {b}x + {c} = 0')
print(f'(x1, x2) = {solve_second_degree_eq(a, b, c)}')  # problem with the first root (cancellation at the numerator)
print(f'(x1, x2) = {solve_second_degree_eq2(a, b, c)}') # problem with the second root (cancellation at the denominator)
print(f'(x1, x2) = {solve_second_degree_eq_stable(a, b, c)}') # more accurate roots

equation: 0.001x^2 + 1000.0x + 0.001 = 0
(x1, x2) = (-9.999894245993346e-07, -999999.999999)
(x1, x2) = (-1.000000000001e-06, -1000010.5755125057)
(x1, x2) = (-1.000000000001e-06, -999999.999999)


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare it with the answer your program gives. The two will not agree perfectly. Why?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [6]:
# function to derivate
f = lambda x: x * (x - 1)

# analytical derivative of f in x0
f_prime = lambda x: 2*x - 1 

def derivate_f(f, x0, delta):
    """Computes the derivative of a function f in x0 according
    to the definition using the increment delta.
    """
    return (f(x0+delta) - f(x0)) / delta

# (a) The results do not agree because of the limited precision of the numerical computation
x0 = 1
delta = 1e-2
print(f'numerical computation : f\'({x0})={derivate_f(f, x0, delta)} w/ delta={delta}')
print(f'analytical computation: f\'({x0})={f_prime(x0)}\n')

# (b) The accuracy does not improve by increasing delta: the smallest error is
# obtained for delta = 10^-8 and the largest error for delta = 10^-14
for delta in [10**-i for i in range(4, 15, 2)]:
    df = derivate_f(f, x0, delta)
    error = abs(df - f_prime(x0))
    print(f'delta={delta:.0e}: f\'({x0})={df},\t error={error}')

numerical computation : f'(1)=1.010000000000001 w/ delta=0.01
analytical computation: f'(1)=1

delta=1e-04: f'(1)=1.0000999999998899,	 error=9.999999988985486e-05
delta=1e-06: f'(1)=1.0000009999177333,	 error=9.99917733279787e-07
delta=1e-08: f'(1)=1.0000000039225287,	 error=3.922528746258536e-09
delta=1e-10: f'(1)=1.000000082840371,	 error=8.284037100736441e-08
delta=1e-12: f'(1)=1.0000889005833413,	 error=8.890058334132256e-05
delta=1e-14: f'(1)=0.9992007221626509,	 error=0.0007992778373491216


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which is known to be $I=\frac{\pi}{2}=1.57079632679...$.

Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a program to compute the integral with $N=100$. How does the result compare to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [7]:
import math
import time

def interval(lower, upper, length):
    """Creates a discrete interval from lower to upper of length samples.
    E.g., if lower=-1, upper=1, length = 4, it returns [-0.75, -0.25, 0.25, 0.75]. 
    """
    delta = (upper - lower) / length
    return [lower + delta*(i + .5) for i in range(length)]

def compute_integral(f, a, b, N):
    """Computes numerically, according to the Riemann definition, the integral
    of the function f in the interval (a,b) using N samples.
    """
    h = (b - a) / N
    return sum([h * f(x) for x in interval(a, b, N)])

# function of the semi circumference
semi_circ = lambda x: math.sqrt(1 - x**2)

# (a) computation of the integral using the Riemann definition w/ 100 samples
N = 100
I = compute_integral(semi_circ, -1, 1, N)
print(f'N={N}: I={I}, error={I-math.pi/2:.2e}\n') # the result is fairly accurate

# (b) find the max value of N to execute in less tha 1s and 60s
exe_time = 0
N_max = 1000000000 # after this value the running time exceeds 1 minute
while exe_time < 60 and N < N_max:
    start = time.time()
    compute_integral(semi_circ, -1, 1, N)
    exe_time = time.time() - start
    print(f'N={N}: {exe_time:.5f}s')
    N *= 10
    
N1 = 1000000   # less than 1s
N2 = 100000000 # less than 60s
I1 = compute_integral(semi_circ, -1, 1, N1)
I2 = compute_integral(semi_circ, -1, 1, N2)
e1 = abs(I1 - math.pi/2)
e2 = abs(I2 - math.pi/2)
print()
print(f'N={N1}: I={I1}, error={e1:.2e}')
print(f'N={N2}: I={I2}, error={e2:.2e}\n')
# Running for up to 60 sec improves the result by 3 orders of magnitude: from 10^-10 to 10^-13

N=100: I=1.5712827762297958, error=4.86e-04

N=100: 0.00000s
N=1000: 0.00000s
N=10000: 0.00500s
N=100000: 0.03403s
N=1000000: 0.38579s
N=10000000: 3.76693s
N=100000000: 37.96444s

N=1000000: I=1.5707963272820449, error=4.87e-10
N=100000000: I=1.5707963267955443, error=6.48e-13

